# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col='City_ID')

# Display sample data
city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,waitangi,-43.9535,-176.5597,11.01,75,60,2.68,NZ,1697147687
1,brookings,44.3114,-96.7984,7.28,97,100,12.35,US,1697147544
2,adamstown,-25.0660,-130.1015,20.91,69,98,5.65,PN,1697146870
3,ribeira grande,38.5167,-28.7000,19.32,84,67,1.79,PT,1697147687
4,thompson,55.7435,-97.8558,13.09,58,40,1.54,CA,1697147482


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    alpha=0.5,
    color="City"
)

# Display the map
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.dropna().loc[
    (city_data_df['Max Temp'] > 20) &
    (city_data_df['Max Temp'] < 25) &
    (city_data_df['Humidity'] > 45) &
    (city_data_df['Humidity'] < 75) &
    (city_data_df['Cloudiness'] > 5) &
    (city_data_df['Cloudiness'] < 75)
]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
8,nuku'alofa,-21.1333,-175.2000,24.19,69,20,4.12,TO,1697147687
53,awjilah,29.1081,21.2869,21.98,55,29,4.55,LY,1697147690
133,carnarvon,-24.8667,113.6333,23.56,59,23,4.19,AU,1697147536
220,nuevo imperial,-13.0754,-76.3172,20.49,73,41,2.63,PE,1697147704
234,portland,45.5234,-122.6762,22.48,63,20,3.09,US,1697146847
253,naze,28.3667,129.4833,22.51,73,74,7.60,JP,1697147707
277,pisco,-13.7000,-76.2167,22.03,60,20,4.63,PE,1697147709
313,waddan,29.1614,16.1390,22.05,55,8,5.36,LY,1697147711
417,columbus,39.9612,-82.9988,24.49,47,40,4.63,US,1697147163


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = (city_data_df
    .copy()
    .dropna()
        .loc[
        (city_data_df['Max Temp'] > 20) &
        (city_data_df['Max Temp'] < 25) &
        (city_data_df['Humidity'] > 45) &
        (city_data_df['Humidity'] < 75) &
        (city_data_df['Cloudiness'] > 5) &
        (city_data_df['Cloudiness'] < 75)
    ][['City', 'Country', 'Lat', 'Lng', 'Humidity']]
)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
8,nuku'alofa,TO,-21.1333,-175.2000,69,
53,awjilah,LY,29.1081,21.2869,55,
133,carnarvon,AU,-24.8667,113.6333,59,
220,nuevo imperial,PE,-13.0754,-76.3172,73,
234,portland,US,45.5234,-122.6762,63,
253,naze,JP,28.3667,129.4833,73,
277,pisco,PE,-13.7000,-76.2167,60,
313,waddan,LY,29.1614,16.1390,55,
417,columbus,US,39.9612,-82.9988,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit": 1,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search


nuku'alofa - nearest hotel: City Hotel
awjilah - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
nuevo imperial - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
naze - nearest hotel: ビジネス旅館畠山
pisco - nearest hotel: La Portada
waddan - nearest hotel: No hotel found
columbus - nearest hotel: Sheraton Columbus Hotel at Capitol Square
westport - nearest hotel: Norwalk Inn and Conference Center
kotido - nearest hotel: La Maison Hotel Annex
zahirabad - nearest hotel: No hotel found
holma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
8,nuku'alofa,TO,-21.1333,-175.2000,69,City Hotel
53,awjilah,LY,29.1081,21.2869,55,No hotel found
133,carnarvon,AU,-24.8667,113.6333,59,No hotel found
220,nuevo imperial,PE,-13.0754,-76.3172,73,No hotel found
234,portland,US,45.5234,-122.6762,63,the Hoxton
253,naze,JP,28.3667,129.4833,73,ビジネス旅館畠山
277,pisco,PE,-13.7000,-76.2167,60,La Portada
313,waddan,LY,29.1614,16.1390,55,No hotel found
417,columbus,US,39.9612,-82.9988,47,Sheraton Columbus Hotel at Capitol Square


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    alpha=0.5,
    color="City",
    hover_cols=["Hotel Name", "Country"]
    
)

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)